In [2]:
!pip install pytesseract


In [4]:
!pip install easyocr

^C


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2025.5.1 requires toolz>=0.10.0, which is not installed.


  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
   ---------------------------------------- 2.9/2.9 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 1.6/1.6 MB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 241.4/241.4 MB 2.4 MB/s eta 0:00:00
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)
   ---------------------------------------- 1.7/1.7 MB 3.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113



In [2]:
import torch

In [ ]:
import tensorflow

In [2]:
!pip uninstall numpy
!pip install numpy

^C


In [1]:
# Install required packages if not already installed
# !pip install opencv-python pytesseract pillow difflib

import cv2
import os
import pytesseract
import pandas as pd
from pathlib import Path
from PIL import Image
import numpy as np
from difflib import SequenceMatcher
import re
from datetime import timedelta
import json

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import

In [ ]:
class SlideDetector:
    def __init__(self, tesseract_path=None, similarity_threshold=0.8, frame_interval=2.0):
        """
        Initialize the SlideDetector
        
        Args:
            tesseract_path (str): Path to tesseract executable (if not in PATH)
            similarity_threshold (float): Threshold for determining if slides are similar (0-1)
            frame_interval (float): Interval in seconds between frame extractions
        """
        if tesseract_path:
            pytesseract.pytesseract.tesseract_cmd = tesseract_path
        
        self.similarity_threshold = similarity_threshold
        self.frame_interval = frame_interval
        self.slide_changes = []
        
    def extract_text_from_frame(self, frame, roi=None):
        """
        Extract text from a video frame using OCR
        
        Args:
            frame: OpenCV frame
            roi: Region of interest (x, y, w, h) to focus OCR on specific area
        
        Returns:
            str: Extracted text
        """
        # Convert frame to PIL Image
        if roi:
            x, y, w, h = roi
            frame = frame[y:y+h, x:x+w]
        
        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_frame)
        
        # Use OCR to extract text
        try:
            text = pytesseract.image_to_string(pil_image, config='--psm 6')
            return text.strip()
        except Exception as e:
            print(f"OCR Error: {e}")
            return ""
    
    def clean_text(self, text):
        """
        Clean extracted text by removing special characters and normalizing
        """
        # Remove extra whitespace and special characters
        cleaned = re.sub(r'[^\w\s]', ' ', text)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()
        return cleaned.lower()
    
    def calculate_text_similarity(self, text1, text2):
        """
        Calculate similarity between two text strings
        
        Returns:
            float: Similarity score (0-1)
        """
        if not text1 or not text2:
            return 0.0
        
        cleaned_text1 = self.clean_text(text1)
        cleaned_text2 = self.clean_text(text2)
        
        return SequenceMatcher(None, cleaned_text1, cleaned_text2).ratio()
    
    def extract_slide_title(self, text):
        """
        Extract the most likely slide title from OCR text
        Assumes title is usually the first few lines or largest text
        """
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        if not lines:
            return "Untitled Slide"
        
        # Take the first non-empty line as title, or combine first few short lines
        title_parts = []
        for line in lines[:3]:  # Check first 3 lines
            if len(line) > 3 and len(line) < 100:  # Reasonable title length
                title_parts.append(line)
            if len(' '.join(title_parts)) > 50:  # Don't make title too long
                break
        
        title = ' '.join(title_parts) if title_parts else lines[0]
        return title[:100]  # Limit title length
    
    def seconds_to_timestamp(self, seconds):
        """
        Convert seconds to MM:SS or HH:MM:SS format
        """
        td = timedelta(seconds=int(seconds))
        total_seconds = int(td.total_seconds())
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        
        if hours > 0:
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        else:
            return f"{minutes:02d}:{seconds:02d}"
    
    def process_video(self, video_path, roi=None, output_file=None):
        """
        Process a single video file to detect slide changes
        
        Args:
            video_path (str): Path to video file
            roi (tuple): Region of interest (x, y, w, h) for title area
            output_file (str): Optional output file path for results
        
        Returns:
            list: List of dictionaries containing timestamp and slide title
        """
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
        
        print(f"Processing video: {os.path.basename(video_path)}")
        
        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")
        
        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        print(f"Video duration: {self.seconds_to_timestamp(duration)}")
        print(f"FPS: {fps:.2f}")
        
        self.slide_changes = []
        previous_text = ""
        frame_number = 0
        
        # Process frames at specified intervals
        while True:
            # Set frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            
            if not ret:
                break
            
            # Calculate timestamp
            timestamp_seconds = frame_number / fps
            
            # Extract text from frame
            current_text = self.extract_text_from_frame(frame, roi)
            
            # Check if this is a new slide
            if previous_text:
                similarity = self.calculate_text_similarity(previous_text, current_text)
                
                if similarity < self.similarity_threshold and len(current_text) > 10:
                    # New slide detected
                    slide_title = self.extract_slide_title(current_text)
                    timestamp = self.seconds_to_timestamp(timestamp_seconds)
                    
                    slide_info = {
                        'timestamp': timestamp,
                        'timestamp_seconds': timestamp_seconds,
                        'slide_title': slide_title,
                        'full_text': current_text
                    }
                    
                    self.slide_changes.append(slide_info)
                    print(f"New slide at {timestamp}: {slide_title}")
            else:
                # First slide
                if len(current_text) > 10:
                    slide_title = self.extract_slide_title(current_text)
                    timestamp = self.seconds_to_timestamp(timestamp_seconds)
                    
                    slide_info = {
                        'timestamp': timestamp,
                        'timestamp_seconds': timestamp_seconds,
                        'slide_title': slide_title,
                        'full_text': current_text
                    }
                    
                    self.slide_changes.append(slide_info)
                    print(f"First slide at {timestamp}: {slide_title}")
            
            previous_text = current_text
            
            # Move to next frame
            frame_number += int(fps * self.frame_interval)
            
            # Progress indicator
            progress = (frame_number / total_frames) * 100
            if frame_number % (int(fps * 10)) == 0:  # Print every 10 seconds
                print(f"Progress: {progress:.1f}%")
        
        cap.release()
        
        # Save results if output file specified
        if output_file:
            self.save_results(output_file, video_path)
        
        print(f"\\nFound {len(self.slide_changes)} slide changes")
        return self.slide_changes
    
    def save_results(self, output_file, video_path):
        """
        Save slide detection results to file
        """
        results = {
            'video_path': video_path,
            'video_name': os.path.basename(video_path),
            'total_slides': len(self.slide_changes),
            'slides': self.slide_changes
        }
        
        # Determine file format based on extension
        if output_file.endswith('.json'):
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
        elif output_file.endswith('.csv'):
            df = pd.DataFrame(self.slide_changes)
            df.to_csv(output_file, index=False, encoding='utf-8')
        else:
            # Default to text format
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"Video: {os.path.basename(video_path)}\\n")
                f.write(f"Total slides: {len(self.slide_changes)}\\n\\n")
                for slide in self.slide_changes:
                    f.write(f"{slide['timestamp']} - {slide['slide_title']}\\n")
        
        print(f"Results saved to: {output_file}")

In [ ]:
class VideoPlaylistProcessor:
    """
    Process multiple videos in a directory/playlist
    """
    
    def __init__(self, tesseract_path=None, similarity_threshold=0.8, frame_interval=2.0):
        self.detector = SlideDetector(tesseract_path, similarity_threshold, frame_interval)
        self.supported_formats = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv', '.webm']
    
    def find_video_files(self, directory):
        """
        Find all video files in the specified directory
        """
        video_files = []
        directory = Path(directory)
        
        if not directory.exists():
            raise FileNotFoundError(f"Directory not found: {directory}")
        
        for file_path in directory.rglob('*'):
            if file_path.suffix.lower() in self.supported_formats:
                video_files.append(str(file_path))
        
        return sorted(video_files)
    
    def process_playlist(self, directory, output_dir=None, roi=None):
        """
        Process all videos in a directory
        
        Args:
            directory (str): Directory containing video files
            output_dir (str): Directory to save results (optional)
            roi (tuple): Region of interest for slide titles
        
        Returns:
            dict: Results for all videos
        """
        video_files = self.find_video_files(directory)
        
        if not video_files:
            print(f"No video files found in {directory}")
            return {}
        
        print(f"Found {len(video_files)} video files to process")
        
        if output_dir:
            output_dir = Path(output_dir)
            output_dir.mkdir(exist_ok=True)
        
        all_results = {}
        
        for i, video_path in enumerate(video_files, 1):
            print(f"\\n{'='*60}")
            print(f"Processing video {i}/{len(video_files)}")
            print(f"{'='*60}")
            
            try:
                video_name = Path(video_path).stem
                
                # Process video
                slide_changes = self.detector.process_video(video_path, roi)
                
                # Save individual results
                if output_dir:
                    output_file = output_dir / f"{video_name}_timestamps.json"
                    self.detector.save_results(str(output_file), video_path)
                
                all_results[video_name] = {
                    'path': video_path,
                    'slides': slide_changes,
                    'total_slides': len(slide_changes)
                }
                
            except Exception as e:
                print(f"Error processing {video_path}: {e}")
                all_results[Path(video_path).stem] = {
                    'path': video_path,
                    'error': str(e),
                    'slides': [],
                    'total_slides': 0
                }
        
        # Save combined results
        if output_dir:
            self.save_combined_results(all_results, output_dir / "all_videos_summary.json")
        
        return all_results
    
    def save_combined_results(self, all_results, output_file):
        """
        Save combined results for all videos
        """
        summary = {
            'total_videos': len(all_results),
            'successful_videos': len([r for r in all_results.values() if 'error' not in r]),
            'total_slides_detected': sum(r['total_slides'] for r in all_results.values()),
            'videos': all_results
        }
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        
        print(f"\\nCombined results saved to: {output_file}")
        
        # Also create a simple CSV summary
        csv_file = str(output_file).replace('.json', '.csv')
        video_summary = []
        for video_name, data in all_results.items():
            video_summary.append({
                'video_name': video_name,
                'total_slides': data['total_slides'],
                'status': 'success' if 'error' not in data else 'error'
            })
        
        df = pd.DataFrame(video_summary)
        df.to_csv(csv_file, index=False)
        print(f"Summary CSV saved to: {csv_file}")

In [ ]:
# Example usage and helper functions

def auto_detect_title_region(video_path, sample_frames=5):
    """
    Automatically detect the region where slide titles are likely located
    by analyzing sample frames
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Sample frames from different parts of the video
    sample_positions = np.linspace(0, total_frames - 1, sample_frames, dtype=int)
    
    height, width = None, None
    
    for pos in sample_positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        ret, frame = cap.read()
        if ret:
            height, width = frame.shape[:2]
            break
    
    cap.release()
    
    if height and width:
        # Assume title is in top 30% of the frame
        roi = (0, 0, width, int(height * 0.3))
        return roi
    
    return None

def display_sample_detection(video_path, roi=None, sample_time=30):
    """
    Display a sample frame with detected text for debugging
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Could not open video")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_pos = int(sample_time * fps)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
    ret, frame = cap.read()
    
    if ret:
        detector = SlideDetector()
        text = detector.extract_text_from_frame(frame, roi)
        title = detector.extract_slide_title(text)
        
        print(f"Sample at {sample_time}s:")
        print(f"Detected title: {title}")
        print(f"Full text: {text[:200]}...")
        
        # Optionally save sample frame
        sample_path = f"sample_frame_{sample_time}s.jpg"
        cv2.imwrite(sample_path, frame)
        print(f"Sample frame saved to: {sample_path}")
    
    cap.release()

# Configuration function
def setup_tesseract_windows():
    """
    Setup function for Windows users to configure Tesseract path
    """
    common_paths = [
        r"C:\Program Files\Tesseract-OCR\tesseract.exe",
        r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe",
        r"C:\Users\{}\AppData\Local\Programs\Tesseract-OCR\tesseract.exe".format(os.getenv('USERNAME'))
    ]
    
    for path in common_paths:
        if os.path.exists(path):
            pytesseract.pytesseract.tesseract_cmd = path
            print(f"Tesseract configured at: {path}")
            return path
    
    print("Tesseract not found in common locations.")
    print("Please install Tesseract OCR from: https://github.com/UB-Mannheim/tesseract/wiki")
    print("Or manually set the path using: pytesseract.pytesseract.tesseract_cmd = 'your_path_here'")
    return None

In [ ]:
# SETUP AND USAGE EXAMPLE

# 1. First, setup Tesseract (required for OCR)
print("Setting up Tesseract OCR...")
tesseract_path = setup_tesseract_windows()

# If tesseract is not found automatically, uncomment and set the path manually:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
# 2. EXAMPLE: Process a single video

# Replace with your video path
video_path = r"The Greatest Portrait Photographer of All Time [WVHw4-62rV4].mp4"

if os.path.exists(video_path):
    print(f"Testing with video: {video_path}")
    
    # Test sample detection first
    print("\\nTesting sample detection...")
    display_sample_detection(video_path, sample_time=30)
    
    # Auto-detect title region
    roi = auto_detect_title_region(video_path)
    print(f"\\nAuto-detected ROI: {roi}")
    
    # Process the video
    detector = SlideDetector(
        similarity_threshold=0.7,  # Adjust based on your needs (0.5-0.9)
        frame_interval=3.0         # Check every 3 seconds
    )
    
    results = detector.process_video(
        video_path=video_path,
        roi=roi,  # Focus on title area
        output_file="single_video_results.json"
    )
    
    print(f"\\nDetected {len(results)} slide changes:")
    for slide in results:
        print(f"  {slide['timestamp']} - {slide['slide_title']}")
else:
    print(f"Video file not found: {video_path}")
    print("Please update the video_path variable with a valid video file.")

In [ ]:
# 3. EXAMPLE: Process multiple videos in a directory

# Replace with your videos directory
videos_directory = r"videos"  # or your specific path like r"C:\Users\username\Videos\Presentations"

processor = VideoPlaylistProcessor(
    similarity_threshold=0.7,
    frame_interval=3.0
)

# Check if directory exists and find videos
if os.path.exists(videos_directory):
    video_files = processor.find_video_files(videos_directory)
    print(f"Found {len(video_files)} video files in {videos_directory}:")
    for i, video in enumerate(video_files, 1):
        print(f"  {i}. {os.path.basename(video)}")
    
    if video_files:
        # Process all videos
        print(f"\\nProcessing all videos...")
        results = processor.process_playlist(
            directory=videos_directory,
            output_dir="slide_detection_results",  # Results will be saved here
            roi=None  # Auto-detect for each video
        )
        
        # Print summary
        print(f"\\n{'='*60}")
        print("PROCESSING SUMMARY")
        print(f"{'='*60}")
        total_slides = sum(data['total_slides'] for data in results.values())
        successful = len([r for r in results.values() if 'error' not in r])
        
        print(f"Videos processed: {len(results)}")
        print(f"Successful: {successful}")
        print(f"Total slides detected: {total_slides}")
        
        print("\\nPer video results:")
        for video_name, data in results.items():
            if 'error' in data:
                print(f"  ❌ {video_name}: ERROR - {data['error']}")
            else:
                print(f"  ✅ {video_name}: {data['total_slides']} slides")
    else:
        print("No video files found in the directory.")
else:
    print(f"Directory not found: {videos_directory}")
    print("Please update the videos_directory variable with the correct path.")

# YouTube Slide Detection Tool

## Overview
This tool processes YouTube presentation videos to automatically detect slide changes using OCR (Optical Character Recognition) and outputs timestamps with slide titles.

## Features
- 🎥 **Batch Processing**: Process multiple videos in a directory
- 🔍 **OCR Detection**: Extract slide titles using Tesseract OCR
- ⏱️ **Timestamp Generation**: Accurate timestamps for each slide change
- 📊 **Multiple Output Formats**: JSON, CSV, and text formats
- 🎯 **ROI Support**: Focus OCR on specific screen regions for better accuracy
- 🔧 **Configurable**: Adjustable similarity thresholds and processing intervals

## Requirements
1. **Python packages**: opencv-python, pytesseract, pillow, pandas, numpy
2. **Tesseract OCR**: Must be installed separately
   - Download from: https://github.com/UB-Mannheim/tesseract/wiki
   - Windows installer available

## Configuration Tips

### Similarity Threshold
- **0.5-0.6**: Very sensitive, detects small changes
- **0.7-0.8**: Balanced, good for most presentations  
- **0.8-0.9**: Less sensitive, only major slide changes

### Frame Interval
- **1-2 seconds**: More accurate but slower processing
- **3-5 seconds**: Faster processing, may miss quick slides
- **5+ seconds**: Fast but might miss slides

### ROI (Region of Interest)
- Use auto-detection or manually specify title area
- Format: (x, y, width, height)
- Example: (0, 0, 1920, 300) for top 300 pixels of 1920px wide video

In [ ]:
# 4. TROUBLESHOOTING AND ADVANCED USAGE

def test_ocr_setup():
    """Test if OCR is working properly"""
    try:
        # Create a simple test image with text
        import numpy as np
        from PIL import Image, ImageDraw, ImageFont
        
        # Create test image
        img = Image.new('RGB', (400, 100), color='white')
        draw = ImageDraw.Draw(img)
        draw.text((10, 30), "Test Slide Title", fill='black')
        
        # Convert to array and test OCR
        img_array = np.array(img)
        text = pytesseract.image_to_string(img_array)
        
        print(f"OCR Test Result: '{text.strip()}'")
        
        if "Test" in text or "Slide" in text:
            print("✅ OCR is working correctly!")
            return True
        else:
            print("❌ OCR might not be working properly")
            return False
            
    except Exception as e:
        print(f"❌ OCR Error: {e}")
        return False

def optimize_settings_for_video(video_path, test_duration=60):
    """
    Test different settings to find optimal configuration for a specific video
    """
    print(f"Optimizing settings for: {os.path.basename(video_path)}")
    
    if not os.path.exists(video_path):
        print("Video file not found!")
        return
    
    # Test different thresholds
    thresholds = [0.6, 0.7, 0.8]
    intervals = [2.0, 3.0, 5.0]
    
    roi = auto_detect_title_region(video_path)
    
    best_config = None
    best_slide_count = 0
    
    for threshold in thresholds:
        for interval in intervals:
            print(f"\\nTesting: threshold={threshold}, interval={interval}s")
            
            detector = SlideDetector(similarity_threshold=threshold, frame_interval=interval)
            
            # Process only first minute for testing
            cap = cv2.VideoCapture(video_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            cap.release()
            
            # Create temporary shortened video processing
            temp_detector = SlideDetector(similarity_threshold=threshold, frame_interval=interval)
            
            try:
                # Quick test - process first few frames
                cap = cv2.VideoCapture(video_path)
                frame_count = 0
                test_frames = int(fps * test_duration)  # Test first 60 seconds
                previous_text = ""
                slide_count = 0
                
                while frame_count < test_frames:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
                    ret, frame = cap.read()
                    if not ret:
                        break
                    
                    current_text = temp_detector.extract_text_from_frame(frame, roi)
                    
                    if previous_text and len(current_text) > 10:
                        similarity = temp_detector.calculate_text_similarity(previous_text, current_text)
                        if similarity < threshold:
                            slide_count += 1
                    
                    previous_text = current_text
                    frame_count += int(fps * interval)
                
                cap.release()
                
                print(f"  Detected {slide_count} slides in first {test_duration}s")
                
                # Prefer configurations that detect a reasonable number of slides
                if 2 <= slide_count <= 15 and slide_count > best_slide_count:
                    best_slide_count = slide_count
                    best_config = {'threshold': threshold, 'interval': interval}
                    
            except Exception as e:
                print(f"  Error: {e}")
    
    if best_config:
        print(f"\\n🎯 Recommended settings:")
        print(f"   similarity_threshold = {best_config['threshold']}")
        print(f"   frame_interval = {best_config['interval']}")
        return best_config
    else:
        print(f"\\n⚠️  Could not determine optimal settings. Try manual adjustment.")
        return {'threshold': 0.7, 'interval': 3.0}

# Test OCR setup
print("Testing OCR setup...")
test_ocr_setup()

# If you want to optimize settings for a specific video:
# optimize_settings_for_video("your_video.mp4")

In [ ]:
# INSTALLATION COMMANDS
# Run these commands in your terminal/command prompt before using the tool:

installation_commands = """
# Install required packages:
pip install opencv-python
pip install pytesseract  
pip install Pillow
pip install numpy

# Install Tesseract OCR (Windows):
# 1. Download from: https://github.com/UB-Mannheim/tesseract/wiki
# 2. Run the installer (tesseract-ocr-w64-setup-5.3.x.exe)
# 3. Add to PATH or set the path manually in the code

# For Linux (Ubuntu/Debian):
# sudo apt install tesseract-ocr

# For macOS:
# brew install tesseract
"""

print("INSTALLATION INSTRUCTIONS:")
print(installation_commands)

# Quick test to see if packages are available
try:
    import cv2
    print("✅ OpenCV imported successfully")
except ImportError as e:
    print(f"❌ OpenCV not available: {e}")

try:
    import pytesseract
    print("✅ pytesseract imported successfully")
except ImportError as e:
    print(f"❌ pytesseract not available: {e}")

try:
    from PIL import Image
    print("✅ Pillow imported successfully")
except ImportError as e:
    print(f"❌ Pillow not available: {e}")

try:
    import numpy as np
    print("✅ NumPy imported successfully")
except ImportError as e:
    print(f"❌ NumPy not available: {e}")